In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv')

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

In [ ]:
# df.drop(['order_id' , 'line_item_type_id'], axis = 1, inplace=True)
df['date'] = df.date.apply(lambda l: pd.Timestamp(l).value)

In [ ]:
df.head()

In [ ]:
#calculating CPM
#calculating the value that the Advertisers Bid for the month of June
# CPM(the value which was the winning bid value) = 
#((revenue of the publisher*100)/revenue_share_percentage)/measurable_impressions)*1000

def weird_division(n, d):
    return n / d if d else 0

df['CPM'] = df.apply(lambda x: weird_division(((x['total_revenue']*100)),x['measurable_impressions'])*1000 , axis=1)


In [ ]:
del df['total_revenue']

In [ ]:
df.tail()

In [ ]:
# we can remove total impressions as well as that is account the same information as measurable impressions 
# also let us try to see if viewable/measurable impressions are corellated to revenue or not 

df['View/measurable'] = df.apply(lambda x: weird_division(x['viewable_impressions'],x['measurable_impressions']) , axis=1)


In [ ]:
#remove the outliers 

# df = df[df['CPM'].between(df['CPM'].quantile(.05), df['CPM'].quantile(.95))]

print(df.shape)
print(df.CPM.quantile(.95))
df = df[df.CPM >= 0]
df = df[df.CPM < df.CPM.quantile(.95)]
print(df.shape)

In [ ]:
df.reset_index(inplace= True)

# Modeling

In [ ]:
df.columns

# XGB

In [ ]:
import xgboost as xgb

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold

X_cols = ['total_impressions', 'viewable_impressions', 'measurable_impressions', 'revenue_share_percent',
          'site_id', 'ad_type_id', 'geo_id', 'device_category_id', 'advertiser_id', 'order_id',
          'line_item_type_id', 'os_id', 'integration_type_id', 'monetization_channel_id', 'ad_unit_id']
y_cols =['CPM']

X_train = df.loc[df.date < pd.Timestamp('06-22-2019').value][X_cols]
y_train = df.loc[df.date < pd.Timestamp('06-22-2019').value][y_cols]
X_test = df.loc[df.date >= pd.Timestamp('06-22-2019').value][X_cols]
y_test = df.loc[df.date >= pd.Timestamp('06-22-2019').value][y_cols]

X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [ ]:
# xgbr = xgb.XGBRegressor(n_estimators=230, learning_rate=0.06, gamma=0, subsample=0.75,#n_estimators=100 #learning_rate=0.08
#                            colsample_bytree=0.6, max_depth=9, random_state=1)
# xgbr.fit(X_train.values,y_train.values)

In [ ]:
from sklearn.metrics import mean_squared_error
# predictions = xgbr.predict(X_test.values)
# mse = mean_squared_error(y_test, predictions)
# print("MSE: %.2f" % mse)

In [ ]:
# predictions[predictions<0] = 0
# predictions_xgb = predictions.copy()
# mean_squared_error(y_test, predictions)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import TimeSeriesSplit, KFold

n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=False, random_state=1)
prediction_xgb_cv = np.zeros(X_test.shape[0])

xgbr = xgb.XGBRegressor(n_estimators=230, learning_rate=0.06, gamma=0, subsample=0.75,#n_estimators=100 #learning_rate=0.08
                           colsample_bytree=0.6, max_depth=9, random_state=1)

for fold_n, (train_index, test_index) in enumerate(folds.split(X_train)):
    print('Fold:', fold_n)
    X_traincv, X_validcv = X_train.iloc[train_index], X_train.iloc[test_index]
    Y_traincv, Y_validcv = y_train.iloc[train_index], y_train.iloc[test_index]

    xgbr.fit(X_traincv.values,Y_traincv.values)
    
    y_pred = xgbr.predict(X_test.values)
    prediction_xgb_cv += y_pred
    
    print(mean_squared_error(y_test, y_pred))
    
prediction_xgb_cv /= n_fold
print('--------------')
mean_squared_error(y_test, prediction_xgb_cv)

In [ ]:
prediction_xgb_cv[prediction_xgb_cv<0] = 0
mean_squared_error(y_test, prediction_xgb_cv)

# H2O

In [ ]:
train = df.loc[df.date < pd.Timestamp('06-22-2019').value].to_csv('train.csv')
test = df.loc[df.date >= pd.Timestamp('06-22-2019').value].to_csv('test.csv')

In [ ]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

In [ ]:
train = h2o.import_file("train.csv")
test = h2o.import_file("test.csv")

In [ ]:
x = ['total_impressions', 'viewable_impressions', 'measurable_impressions', 'revenue_share_percent',
          'site_id', 'ad_type_id', 'geo_id', 'device_category_id', 'advertiser_id', 'order_id',
          'line_item_type_id', 'os_id', 'integration_type_id', 'monetization_channel_id', 'ad_unit_id']
y = "CPM"

aml = H2OAutoML(max_models=50, seed=1, stopping_metric='MSE', nfolds = 0)
aml.train(x=x, y=y, training_frame=train)

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [ ]:
preds = aml.predict(test)

In [ ]:
mean_squared_error(y_test.values, h2o.as_list(preds))

In [ ]:
predictions_h2o = np.array(h2o.as_list(preds).values).reshape(1, len(preds))[0]
predictions_h2o[predictions_h2o<0] = 0

# LGB

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split, cross_val_score, KFold

In [ ]:
cat_cols = ['measurable_impressions', 'revenue_share_percent',
          'site_id', 'ad_type_id', 'geo_id', 'device_category_id', 'advertiser_id', 'order_id',
          'line_item_type_id', 'os_id', 'integration_type_id', 'monetization_channel_id', 'ad_unit_id']
features = cat_cols + ['total_impressions', 'viewable_impressions',]

In [ ]:
# from sklearn.preprocessing import StandardScaler

# train = df.loc[df.date < pd.Timestamp('06-22-2019').value]
# test = df.loc[df.date >= pd.Timestamp('06-22-2019').value]

# scaler = StandardScaler()
# scaler.fit(train[features])
# train[features] = pd.DataFrame(scaler.transform(train[features]), columns= features)
# test[features] = pd.DataFrame(scaler.transform(test[features]), columns= features)



train = df.loc[df.date < pd.Timestamp('06-22-2019').value]
test = df.loc[df.date >= pd.Timestamp('06-22-2019').value]

train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

for col in cat_cols:
    train[col] = train[col].astype('category')
    test[col] = test[col].astype('category')
    
train, valid = train_test_split(train, test_size=0.05, shuffle=True, random_state=42)

In [ ]:
params = {
    'objective': 'regression',
    'n_estimators': 500,
    'learning_rate': 0.05,
    'num_leaves': 250,
    'metric': 'mse',
    
}

In [ ]:
lgb_train = lgb.Dataset(train[features], train["CPM"])
lgb_valid = lgb.Dataset(valid[features], valid["CPM"])

gbm = lgb.train(params, lgb_train, 25000, 
    valid_sets=[lgb_train, lgb_valid],
    early_stopping_rounds=100, verbose_eval=1000)


In [ ]:
pred = gbm.predict(test[features])
y_test = test['CPM']
pred[pred<0]=0
predictions_lgb = pred.copy()
mean_squared_error(y_test, pred)

In [ ]:
# gbmcv = lgb.LGBMRegressor(objective = 'regression',  # gbm никак далее не использую, но смотрю на важность фич
#                             max_depth = 9,#было  3
#                             colsample_bytre = 0.8,
#                             subsample = 0.8,
#                             learning_rate = 0.1,
#                             n_estimators = 300, random_state = 1)

# params = {
#     'objective': 'regression',
#     'n_estimators': 500,
#     'learning_rate': 0.05,
#     'num_leaves': 250,
#     'metric': 'mse',
    
# }

# n_fold = 10
# folds = KFold(n_splits=n_fold, shuffle=False, random_state=1)

# X_train = df.loc[df.date < pd.Timestamp('06-22-2019').value][X_cols]
# y_train = df.loc[df.date < pd.Timestamp('06-22-2019').value][y_cols]
# X_test = df.loc[df.date >= pd.Timestamp('06-22-2019').value][X_cols]
# y_test = df.loc[df.date >= pd.Timestamp('06-22-2019').value][y_cols]

# X_train.fillna(0, inplace=True)
# X_test.fillna(0, inplace=True)

# prediction_lgb_cv = np.zeros(X_test.shape[0])

# for fold_n, (train_index, test_index) in enumerate(folds.split(X_train)):
#     print('Fold:', fold_n)
#     X_traincv, X_validcv = X_train.iloc[train_index], X_train.iloc[test_index]
#     Y_traincv, Y_validcv = y_train.iloc[train_index], y_train.iloc[test_index]
    
#     lgb_train = lgb.Dataset(X_traincv, Y_traincv)
#     lgb_valid = lgb.Dataset(X_validcv, Y_validcv)

#     gbmcv = lgb.train(params, lgb_train, 15000, 
#     valid_sets=[lgb_train, lgb_valid],
#     early_stopping_rounds=100, verbose_eval=50)

# #     gbmcv.fit(X_traincv, Y_traincv, eval_set=[(X_validcv, Y_validcv)], eval_metric='mse', early_stopping_rounds=25)
    
#     y_pred = gbmcv.predict(X_test)
#     prediction_lgb_cv += y_pred
    
#     print(mean_squared_error(y_test, y_pred))
    
# prediction_lgb_cv /= n_fold
# print('--------------')
# mean_squared_error(y_test, prediction_lgb_cv)

In [ ]:
# prediction_lgb_cv[prediction_lgb_cv<0] = 0
# mean_squared_error(y_test, prediction_lgb_cv)

# CAT

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.ensemble import ExtraTreesRegressor
from catboost import CatBoostRegressor

train = df.loc[df.date < pd.Timestamp('06-22-2019').value]
test = df.loc[df.date >= pd.Timestamp('06-22-2019').value]

# rfr_model = RandomForestRegressor(n_estimators=500)
# etr_model = ExtraTreesRegressor(n_estimators=500)
cat_model = CatBoostRegressor(iterations=15500, cat_features=cat_cols, verbose=500)

In [ ]:
# rfr_model.fit(train[features], train["CPM"])
# etr_model.fit(train[features], train["CPM"])
cat_model.fit(train[features], train["CPM"])

In [ ]:
test = df.loc[df.date >= pd.Timestamp('06-22-2019').value][features]
# rfr_predict = rfr_model.predict(test)
# etr_predict = etr_model.predict(test)
predictions_cat = cat_model.predict(test)
predictions_cat[predictions_cat<0] = 0

# mean_squared_error(y_test, rfr_predict), mean_squared_error(y_test, etr_predict)
mean_squared_error(y_test, predictions_cat)

In [ ]:
# cat_model2 = CatBoostRegressor(iterations=2500,random_seed = 1)

# X_train = df.loc[df.date < pd.Timestamp('06-22-2019').value][X_cols]
# y_train = df.loc[df.date < pd.Timestamp('06-22-2019').value][y_cols]
# X_test = df.loc[df.date >= pd.Timestamp('06-22-2019').value][X_cols]
# y_test = df.loc[df.date >= pd.Timestamp('06-22-2019').value][y_cols]

# prediction_cat_cv = np.zeros(X_test.shape[0])

# for fold_n, (train_index, test_index) in enumerate(folds.split(X_train)):
#     print('Fold:', fold_n)
#     X_traincv, X_validcv = X_train.iloc[train_index], X_train.iloc[test_index]
#     Y_traincv, Y_validcv = y_train.iloc[train_index], y_train.iloc[test_index]

#     cat_model2.fit(X_traincv, Y_traincv)
    
#     y_pred = cat_model2.predict(X_test)
#     prediction_cat_cv += y_pred
    
#     print(mean_squared_error(y_test, y_pred))
    
# prediction_cat_cv /= n_fold
# print('--------------')
# mean_squared_error(y_test, prediction_cat_cv)

In [ ]:
# prediction_cat_cv[prediction_cat_cv<0] = 0
# mean_squared_error(y_test, prediction_cat_cv)

In [ ]:
import operator
res = {}
for i in np.arange(0,3,step=0.1):
    for j in np.arange(0,3,step=0.1):
        for k in np.arange(0,3,step=0.1):
            for l in np.arange(0,3,step=0.1):
                if i!=0 or j !=0 or k!=0 or l!=0:
                    mse = mean_squared_error(y_test, (i*predictions_h2o+j*predictions_lgb+k*predictions_cat+l*prediction_xgb_cv)/(i+j+k+l))
                    res[(i,j,k,l)] = mse

In [ ]:
min(res.items(), key=operator.itemgetter(1))[0], res[min(res.items(), key=operator.itemgetter(1))[0]]

In [ ]:
i, j, k, l = min(res.items(), key=operator.itemgetter(1))[0]
mean_squared_error(y_test, (i*predictions_h2o+j*predictions_lgb+k*predictions_cat+l*prediction_xgb_cv)/(i+j+k+l))

In [ ]:
import operator
res = {}
for i in np.arange(0,5,step=0.1):
    for j in np.arange(0,5,step=0.1):
        for k in np.arange(0,5,step=0.1):
            if i!=0 or j !=0 or k!=0:
                mse = mean_squared_error(y_test, (i*predictions_lgb+j*predictions_cat)/(i+j+k))
                res[(i,j,k)] = mse

In [ ]:
min(res.items(), key=operator.itemgetter(1))[0], res[min(res.items(), key=operator.itemgetter(1))[0]]

In [ ]:
i, j, k = min(res.items(), key=operator.itemgetter(1))[0]
mean_squared_error(y_test, (i*predictions_lgb+j*predictions_cat)/(i+j+k))

# Hochu zachet & podarok!